In [1]:
import json
import re
import unicodedata

In [2]:
import sys
!{sys.executable} -m pip install -U vllm -v --timeout 100


Using pip 25.0.1 from /opt/conda/lib/python3.11/site-packages/pip (python 3.11)
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

MODEL_DIR = "/home/dataset-assist-0/data/models/Qwen2.5-14B-Instruct"  # 改成你真实的模型目录（有config.json）

from vllm import LLM
llm = LLM(
    model=MODEL_DIR,
    tensor_parallel_size=2,   # 你2张A800，建议=2；如果出错先改成1
    dtype="auto",
    max_model_len=32768,
)

print("llm loaded")


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 12-29 17:03:02 [utils.py:253] non-default args: {'max_model_len': 32768, 'tensor_parallel_size': 2, 'disable_log_stats': True, 'model': '/home/dataset-assist-0/data/models/Qwen2.5-14B-Instruct'}
INFO 12-29 17:03:02 [model.py:514] Resolved architecture: Qwen2ForCausalLM
INFO 12-29 17:03:02 [model.py:1661] Using max model len 32768
INFO 12-29 17:03:03 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=230256) INFO 12-29 17:03:03 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='/home/dataset-assist-0/data/models/Qwen2.5-14B-Instruct', speculative_config=None, tokenizer='/home/dataset-assist-0/data/models/Qwen2.5-14B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=Fa

libibverbs: Warning: couldn't open config directory '/etc/libibverbs.d'.
libibverbs: Warning: couldn't open config directory '/etc/libibverbs.d'.


(EngineCore_DP0 pid=230256) (Worker_TP0 pid=230270) INFO 12-29 17:03:05 [gpu_model_runner.py:3562] Starting to load model /home/dataset-assist-0/data/models/Qwen2.5-14B-Instruct...


(EngineCore_DP0 pid=230256) (Worker_TP1 pid=230272) /opt/conda/lib/python3.11/site-packages/tvm_ffi/_optional_torch_c_dlpack.py:174: UserWarning: Failed to JIT torch c dlpack extension, EnvTensorAllocator will not be enabled.
(EngineCore_DP0 pid=230256) (Worker_TP1 pid=230272) We recommend installing via `pip install torch-c-dlpack-ext`
(EngineCore_DP0 pid=230256) (Worker_TP1 pid=230272)   warnings.warn(
(EngineCore_DP0 pid=230256) (Worker_TP0 pid=230270) /opt/conda/lib/python3.11/site-packages/tvm_ffi/_optional_torch_c_dlpack.py:174: UserWarning: Failed to JIT torch c dlpack extension, EnvTensorAllocator will not be enabled.
(EngineCore_DP0 pid=230256) (Worker_TP0 pid=230270) We recommend installing via `pip install torch-c-dlpack-ext`
(EngineCore_DP0 pid=230256) (Worker_TP0 pid=230270)   warnings.warn(


(EngineCore_DP0 pid=230256) (Worker_TP0 pid=230270) INFO 12-29 17:03:07 [cuda.py:351] Using FLASH_ATTN attention backend out of potential backends: ('FLASH_ATTN', 'FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION')


Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]m 
Loading safetensors checkpoint shards:  12% Completed | 1/8 [00:01<00:12,  1.80s/it]
Loading safetensors checkpoint shards:  25% Completed | 2/8 [00:02<00:06,  1.07s/it]
Loading safetensors checkpoint shards:  38% Completed | 3/8 [00:03<00:06,  1.24s/it]
Loading safetensors checkpoint shards:  50% Completed | 4/8 [00:29<00:44, 11.04s/it]
Loading safetensors checkpoint shards:  62% Completed | 5/8 [00:31<00:23,  7.68s/it]
Loading safetensors checkpoint shards:  75% Completed | 6/8 [00:33<00:11,  5.66s/it]
Loading safetensors checkpoint shards:  88% Completed | 7/8 [00:35<00:04,  4.38s/it]
Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:36<00:00,  3.52s/it]
Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:36<00:00,  4.59s/it]
(EngineCore_DP0 pid=230256) (Worker_TP0 pid=230270) 


(EngineCore_DP0 pid=230256) (Worker_TP0 pid=230270) INFO 12-29 17:03:44 [default_loader.py:308] Loading weights took 36.79 seconds
(EngineCore_DP0 pid=230256) (Worker_TP0 pid=230270) INFO 12-29 17:03:44 [gpu_model_runner.py:3659] Model loading took 13.9282 GiB memory and 38.528860 seconds
(EngineCore_DP0 pid=230256) (Worker_TP0 pid=230270) INFO 12-29 17:03:51 [backends.py:643] Using cache directory: /home/batchcom/.cache/vllm/torch_compile_cache/8dad701a84/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=230256) (Worker_TP0 pid=230270) INFO 12-29 17:03:51 [backends.py:703] Dynamo bytecode transform time: 6.91 s
(EngineCore_DP0 pid=230256) (Worker_TP1 pid=230272) (EngineCore_DP0 pid=230256) INFO 12-29 17:04:04 [backends.py:226] Directly load the compiled graph(s) for compile range (1, 8192) from the cache, took 9.085 s
(Worker_TP0 pid=230270) INFO 12-29 17:04:04 [backends.py:226] Directly load the compiled graph(s) for compile range (1, 8192) from the cache, took 9.042 s
(

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:02<00:00, 17.76it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 19.59it/s]


(EngineCore_DP0 pid=230256) (Worker_TP0 pid=230270) INFO 12-29 17:04:11 [custom_all_reduce.py:216] Registering 8342 cuda graph addresses
(EngineCore_DP0 pid=230256) (Worker_TP1 pid=230272) INFO 12-29 17:04:11 [custom_all_reduce.py:216] Registering 8342 cuda graph addresses
(EngineCore_DP0 pid=230256) (Worker_TP0 pid=230270) INFO 12-29 17:04:11 [gpu_model_runner.py:4587] Graph capturing finished in 6 secs, took 0.82 GiB
(EngineCore_DP0 pid=230256) INFO 12-29 17:04:11 [core.py:259] init engine (profile, create kv cache, warmup model) took 26.90 seconds
INFO 12-29 17:04:12 [llm.py:360] Supported tasks: ['generate']
llm loaded


In [4]:
!pip -q install -U pymupdf


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [5]:
import sys
!{sys.executable} -m pip -q install -U pandas


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip -q install -U transformers


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [7]:
from pathlib import Path

INPUT_DIR = Path("/home/dataset-assist-0/annual-reports")   # 放年报PDF的目录
OUTPUT_DIR = Path("/home/dataset-assist-0/annual-reports_output")  # 输出目录（改成你的）
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

pdf_paths = sorted(INPUT_DIR.glob("*.pdf"))
len(pdf_paths), [p.name for p in pdf_paths[:5]]


(8,
 ['NVIDIA-2018-Annual-Report.pdf',
  'NVIDIA-2019-Annual-Report.pdf',
  'NVIDIA-2020-Annual-Report.pdf',
  'NVIDIA-2021-Annual-Report.pdf',
  'NVIDIA-2022-Annual-Report.pdf'])

In [8]:
import fitz  # pymupdf

def pdf_to_text(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    parts = []
    for page in doc:
        t = page.get_text("text")
        if t and t.strip():
            parts.append(t)
    return "\n\n".join(parts)


In [11]:
from transformers import AutoTokenizer
from vllm import SamplingParams

# 你已加载过 llm 的话，这里只要保证 MODEL_DIR 和 tokenizer 有
MODEL_DIR = "/home/dataset-assist-0/data/models/Qwen2.5-14B-Instruct"  # 改成你的模型目录
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)

MAX_MODEL_LEN = 32768
RESERVE_FOR_OUTPUT = 1500
def clean_text(t: str) -> str:
    # 统一字符（全角/半角等）
    t = unicodedata.normalize("NFKC", t)

    # 去控制字符（很多“乱码感”来自这里）
    t = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", " ", t)

    # 去掉“纯数字行”（页码/索引常见，会触发重复）
    t = "\n".join(line for line in t.splitlines() if not re.fullmatch(r"\s*\d+\s*", line))

    # 合并过多空行
    t = re.sub(r"\n{3,}", "\n\n", t)
    return t

def chunk_by_tokens(text, chunk_tokens=6000, overlap_tokens=300):
    ids = tokenizer.encode(text)
    step = chunk_tokens - overlap_tokens
    chunks = []
    for start in range(0, len(ids), step):
        end = min(start + chunk_tokens, len(ids))
        chunks.append(tokenizer.decode(ids[start:end]))
        if end >= len(ids):
            break
    return chunks

def build_one_shot_prompt(annual_text: str, instruction: str) -> str:
    return f"""You are a rigorous equity research analyst. Output must be in English. Use ONLY the provided text.

【INSTRUCTION】
{instruction}

【SOURCE TEXT】
{annual_text}
"""
def extract_first_json(text: str) -> dict:
    # 取第一个 {...}，后面即使模型复读/“请确认”，也会被丢弃
    m = re.search(r"\{.*\}", text, flags=re.S)
    if not m:
        raise ValueError("No JSON object found in output")
    return json.loads(m.group(0))

def dedup_keep_order(items):
    seen = set()
    out = []
    for x in items:
        if not isinstance(x, str):
            continue
        x2 = x.strip()
        if x2 and x2 not in seen:
            seen.add(x2)
            out.append(x2)
    return out

def normalize_card(card: dict) -> dict:
    for k in ["facts", "major_events", "risks", "quotes"]:
        if k in card and isinstance(card[k], list):
            card[k] = dedup_keep_order(card[k])
    return card

def build_map_prompts(chunks):
    prompts = []
    for i, ch in enumerate(chunks, 1):
        prompts.append(f"""
You will receive a chunk of an annual report (English). Extract information strictly from the text.
Output ONE JSON object only (no markdown, no explanations, no confirmations).
After the JSON, output the marker <END_JSON> and stop.

JSON schema:
- chunk_id: "c{i:03d}"
- facts: 3-8 factual bullets (prefer numbers when present)
- financial_metrics: array of objects like {{"name":"Revenue","value":"...","period":"..."}}
- major_events: 0-5 items
- risks: 0-5 items
- quotes: 2-5 short verbatim quotes from the text supporting key facts/numbers

SOURCE TEXT:
{ch}
<END_JSON>
""".strip())
    return prompts


def build_reduce_prompt(cards, instruction: str) -> str:
    return f"""
You must write the final output in English only.

You will receive multiple JSON cards extracted from an annual report.
Follow the INSTRUCTION strictly and write the final research summary.

Hard rules:
- Use ONLY facts/numbers/events present in the cards. Do not add new information.
- If a required item is missing, write: 'Not disclosed in the provided text.'
- Every numeric datapoint or specific claim must include chunk_id sources, e.g., (source: c003,c007).
- Avoid repetition. Do not restate the same point across sections.
- Each heading should have at most 5 bullet points.

INSTRUCTION:
{instruction}

CARDS:
{chr(10).join(cards)}
""".strip()




In [12]:
ONE_SHOT_SAMPLING = SamplingParams(
    temperature=0.2,
    max_tokens=1100,
    repetition_penalty=1.08,
)

MAP_SAMPLING = SamplingParams(
    temperature=0.0,
    max_tokens=650,              # 不要太大，避免跑飞
    repetition_penalty=1.15,
    stop=["<END_JSON>"],
)

MAP_RETRY_SAMPLING = SamplingParams(
    temperature=0.0,
    max_tokens=650,
    repetition_penalty=1.25,
    stop=["<END_JSON>"],
)

REDUCE_SAMPLING = SamplingParams(
    temperature=0.15,
    max_tokens=1200,
    repetition_penalty=1.12,
)

def generate_report_for_text(annual_text: str, llm, instruction: str,
                             chunk_tokens=6000, overlap_tokens=300):
    total_tokens = len(tokenizer.encode(annual_text))
    can_one_shot = total_tokens <= (MAX_MODEL_LEN - RESERVE_FOR_OUTPUT)

    if can_one_shot:
        prompt = build_one_shot_prompt(annual_text, instruction)
        out = llm.generate([prompt], ONE_SHOT_SAMPLING)[0].outputs[0].text
        return out, {"mode": "one_shot", "total_tokens": total_tokens, "chunks": 0}

    chunks = chunk_by_tokens(annual_text, chunk_tokens=chunk_tokens, overlap_tokens=overlap_tokens)
    map_prompts = build_map_prompts(chunks)

    map_outputs = llm.generate(map_prompts, MAP_SAMPLING)

    cards_dicts = []
    bad = 0

    for idx, o in enumerate(map_outputs, 1):
        raw = o.outputs[0].text
        try:
            card = extract_first_json(raw)
        except Exception:
            # 重试一次（更严格）
            retry_raw = llm.generate([map_prompts[idx-1]], MAP_RETRY_SAMPLING)[0].outputs[0].text
            try:
                card = extract_first_json(retry_raw)
            except Exception:
                bad += 1
                card = {
                    "chunk_id": f"c{idx:03d}",
                    "facts": [],
                    "financial_metrics": [],
                    "major_events": [],
                    "risks": [],
                    "quotes": [],
                    "missing": ["该分块抽取失败"]
                }

        card = normalize_card(card)
        card.setdefault("chunk_id", f"c{idx:03d}")
        cards_dicts.append(card)

    # Reduce 只喂“干净 JSON”
    cards_clean_json = [json.dumps(c, ensure_ascii=False) for c in cards_dicts]

    reduce_prompt = build_reduce_prompt(cards_clean_json, instruction)
    report = llm.generate([reduce_prompt], REDUCE_SAMPLING)[0].outputs[0].text

    return report, {"mode": "map_reduce", "total_tokens": total_tokens, "chunks": len(chunks), "bad_cards": bad}



In [13]:
from tqdm.auto import tqdm
import json

instruction = (
     "You are writing for professional investors. Produce an English equity research summary "
    "based ONLY on the provided annual report text. Style: concise, information-dense, "
    "sell-side/buy-side research note tone (no marketing language).\n\n"

    "Length: 300–450 words. Use short paragraphs and bullets where appropriate.\n\n"

    "Use the following headings EXACTLY:\n"
    "1) Investment View (Neutral)\n"
    "2) Key Drivers & Operating Trends (3–6 bullets)\n"
    "3) Financial Snapshot (table-like bullets; include YoY/segment/geography if stated)\n"
    "4) Capital Allocation & Balance Sheet\n"
    "5) Guidance / Outlook (if disclosed)\n"
    "6) Risks & Watch Items (3–6 bullets)\n\n"

    "Hard rules:\n"
    "- No hallucinations: do NOT invent numbers, guidance, customers, products, events, or causality.\n"
    "- If a requested item is not explicitly disclosed, write: 'Not disclosed in the provided text.'\n"
    "- Every numeric datapoint or specific claim must include a source tag with chunk_id "
    "e.g., (source: c012) or (source: c012,c019).\n"
    "- Avoid repetition: do not restate the same point across sections.\n"
    "- Prefer precise financial language (e.g., revenue, gross margin, operating income, "
    "FCF, capex, working capital, deferred revenue) when the source text supports it.\n"
)

results = []

for pdf_path in tqdm(pdf_paths):
    out_txt = OUTPUT_DIR / (pdf_path.stem + "_short_report.txt")
    out_meta = OUTPUT_DIR / (pdf_path.stem + "_meta.json")

    # 已有结果就跳过（可删掉这段让它覆盖重跑）
    if out_txt.exists() and out_meta.exists():
        continue

    annual_text = clean_text(pdf_to_text(str(pdf_path)))
    report, meta = generate_report_for_text(annual_text, llm, instruction)

    out_txt.write_text(report, encoding="utf-8")
    out_meta.write_text(json.dumps({"file": pdf_path.name, **meta}, ensure_ascii=False, indent=2), encoding="utf-8")
    results.append({"file": pdf_path.name, **meta})

results[:3], len(results)


  0%|          | 0/8 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (162328 > 131072). Running this sequence through the model will result in indexing errors

Adding requests: 100%|██████████| 29/29 [00:00<00:00, 87.38it/s]

Processed prompts: 100%|██████████| 29/29 [00:28<00:00,  1.03it/s, est. speed input: 6202.58 toks/s, output: 417.95 toks/s]

Adding requests: 100%|██████████| 1/1 [00:00<00:00, 67.99it/s]

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.99s/it, est. speed input: 1539.32 toks/s, output: 65.38 toks/s]

Adding requests: 100%|██████████| 1/1 [00:00<00:00, 69.18it/s]

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.64s/it, est. speed input: 1691.26 toks/s, output: 65.50 toks/s]

Adding requests: 100%|██████████| 1/1 [00:00<00:00, 62.06it/s]

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.67s/it, est. speed input: 1316.91 toks/s, output: 65.58 toks/s]

Adding requests: 100%|███

([{'file': 'NVIDIA-2018-Annual-Report.pdf',
   'mode': 'map_reduce',
   'total_tokens': 162328,
   'chunks': 29,
   'bad_cards': 0},
  {'file': 'NVIDIA-2019-Annual-Report.pdf',
   'mode': 'map_reduce',
   'total_tokens': 119867,
   'chunks': 21,
   'bad_cards': 3},
  {'file': 'NVIDIA-2020-Annual-Report.pdf',
   'mode': 'map_reduce',
   'total_tokens': 158432,
   'chunks': 28,
   'bad_cards': 4}],
 8)

In [14]:
import pandas as pd
df = pd.DataFrame(results)
df


,file,mode,total_tokens,chunks,bad_cards
0,NVIDIA-2018-Annual-Report.pdf,map_reduce,162328,29,0
1,NVIDIA-2019-Annual-Report.pdf,map_reduce,119867,21,3
2,NVIDIA-2020-Annual-Report.pdf,map_reduce,158432,28,4
3,NVIDIA-2021-Annual-Report.pdf,map_reduce,129032,23,4
4,NVIDIA-2022-Annual-Report.pdf,map_reduce,161942,29,3
5,NVIDIA-2023-Annual-Report.pdf,map_reduce,140083,25,2
6,NVIDIA-2024-Annual-Report.pdf,map_reduce,149451,27,2
7,NVIDIA-2025-Annual-Report.pdf,map_reduce,158234,28,0
